In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import ase
import time
import glob
import numpy as np
import ase.io as aio
import scipy.io as sio
import multiprocessing as mp
import poremks.porosity as pore
from toolz.curried import pipe, curry
import poremks.grid_generator as gen
from poremks.helpers import write2vtk

## For a periodic 3D Zeolite

In [3]:
cif = "MFI.cif"
fname = os.path.join("input3dSlabs", cif)
rep = [1]*3
radii={"Si":1.35, "O": 1.35}
len_pixel=10

## For an amorphous hydrocarbon

In [4]:
cif = "EFK_50A_0.80.xyz"
fname = os.path.join("hydrocarbons", "EFK_50A_0.80.xyz")
rep = [1]*3
radii={"C":1.7, "H": 1.1, "O": 1.52}
len_pixel=10

In [5]:
%%time
atom = aio.read(fname).repeat(rep)

S = gen.grid_maker(atom,
                   len_pixel=len_pixel,
                   radii=radii.copy(),
                   full=False,
                   fft=False)[0]

padval = ((1, 1), (1, 1), (0, 0))
S_dgrid = pipe(S,
               lambda s: np.pad(s, padval, 'constant', constant_values=0),
               lambda s: pore.dgrid(s, len_pixel))

pld = pore.get_pld(S_dgrid)
lcd = pore.get_lcd(S_dgrid)

print(pld)
print(lcd)

4.1640625
16.176525115966797
CPU times: user 47.3 s, sys: 11.7 s, total: 59 s
Wall time: 54.7 s


In [ ]:
write2vtk(S, "%s.vtk" % cif[:-4])